In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [4]:
import pandas as pd
idx = pd.IndexSlice

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression

from datetime import timedelta

%aimport trans.data
%aimport trans.gtrans
%aimport trans.reg
%aimport trans.regpipe

from trans.data import GetData
gd = GetData()
from trans.gtrans import *
from trans.reg import Reg, RegAttr
from trans.regpipe import RegPipe

pctOnlyTrans = GenSelectAttrsTransformer(['Pct'], dropSingle=False )

## Get the raw data

In [5]:
%aimport trans.data
raw_df = gd.combine_data(['FB', 'AAPL', 'AMZN', 
                           'NFLX', 'GOOG', 'SPY'])
raw_df.head()

Open                                          High            \
             FB      AAPL     AMZN NFLX GOOG         SPY   FB      AAPL   
Date                                                                      
2000-01-03  NaN  3.745536  81.5000  NaN  NaN  148.250000  NaN  4.017857   
2000-01-04  NaN  3.866071  85.3750  NaN  NaN  143.531204  NaN  3.950893   
2000-01-05  NaN  3.705357  70.5000  NaN  NaN  139.937500  NaN  3.948661   
2000-01-06  NaN  3.790179  71.3125  NaN  NaN  139.625000  NaN  3.821429   
2000-01-07  NaN  3.446429  67.0000  NaN  NaN  140.312500  NaN  3.607143   

                            ...    Adj Close                       Volume  \
               AMZN NFLX    ...         AMZN NFLX GOOG         SPY     FB   
Date                        ...                                             
2000-01-03  89.5625  NaN    ...      89.3750  NaN  NaN  103.807343    NaN   
2000-01-04  91.5000  NaN    ...      81.9375  NaN  NaN   99.747833    NaN   
2000-01-05  75.1250  NaN    ...      69.7500  NaN  NaN   99.926254    NaN   
2000-01-06  72.6875  NaN    ...      65.5625  NaN  NaN   98.320305    NaN   
2000-01-07  70.5000  NaN    ...      69.5625  NaN  NaN  104.030365    NaN   

                                                         
                   AAPL        AMZN NFLX GOOG       SPY  
Date                                                     
2000-01-03  133949200.0  16117600.0  NaN  NaN   8164300  
2000-01-04  128094400.0  17487400.0  NaN  NaN   8089800  
2000-01-05  194580400.0  38457400.0  NaN  NaN  12177900  
2000-01-06  191993200.0  18752000.0  NaN  NaN   6227200  
2000-01-07  115183600.0  10505400.0  NaN  NaN   8066500  

[5 rows x 36 columns]

## Define featUn transformer: compute Pct and append to Adj Close

In [4]:
raw_df.to_csv("/tmp/raw_df.csv")

In [6]:
pipe_close = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True )
                      )   

pipe_pct   = make_pipeline(GenSelectAttrsTransformer(['Adj Close'], dropSingle=True ), 
                         pctTrans,
                      )

featUn = GenDataFrameFeatureUnion( [ ("Adj Close", pipe_close),
                                    ("Pct", pipe_pct)
                                   ] )

In [7]:
pipe_pct_only   = make_pipeline(GenSelectAttrsTransformer(['Pct'], dropSingle=False ) ) 

## Create pipeline to prepare data for regression
### NOTE: should really RestrictToCalendar BEFORE doing pctTrans

In [8]:
pipe_nn = make_pipeline( featUn,
                         DatetimeIndexTransformer("Dt"),
                         pctOnlyTrans,
                         RestrictToCalendarColTransformer( ("Pct", "SPY")),
                         RestrictToNonNullTransformer("all"),
                         # FillNullTransformer(method="bfill")
                       )
pct_df = pipe_nn.fit_transform(raw_df)
pct_df.head()

Pct                                  
                AAPL      AMZN  FB GOOG NFLX       SPY
Dt                                                    
2000-01-04 -0.084310 -0.083217 NaN  NaN  NaN -0.039106
2000-01-05  0.014634 -0.148741 NaN  NaN  NaN  0.001789
2000-01-06 -0.086538 -0.060036 NaN  NaN  NaN -0.016071
2000-01-07  0.047368  0.061010 NaN  NaN  NaN  0.058076
2000-01-10 -0.017588 -0.005391 NaN  NaN  NaN  0.003431

## Do a rolling regression on the dataframe with prepared data

In [8]:
pct_df.to_csv("/tmp/pct_df.csv")

## Date arithmetic:
### Keep in mind:
- timedelta (for days)
- monthdelta (affects months, use last day of month if non-calendar day)
- dateutils.relativedelta

## Test  basic Reg object

In [9]:
import dateutil.parser as dup
import dateutil.relativedelta as rd

regWindow = rd.relativedelta(months=+6)
regStep   = rd.relativedelta(weeks=+4)

regStart = dup.parse("01/01/2000")
regEnd   = dup.parse("12/29/2017")

In [10]:
ra = Reg(pct_df)
ma = ra.modelCols( [ idx["Pct", "SPY"]])
ma

beta_df = ra.rollingModelAll( *ma, #idx["Pct", "AAPL"],
                 regStart,
                 # pd.to_datetime("04/14/2000", infer_datetime_format=True),
                 regEnd,
                 #rd.relativedelta(weeks=+4)
                 regWindow,
                 stepTimeDelta=regStep
                 # timedelta(weeks=4)
                             
            )
beta_df.tail()

([('Pct', 'SPY')],
 [('Pct', 'AAPL'),
  ('Pct', 'AMZN'),
  ('Pct', 'FB'),
  ('Pct', 'GOOG'),
  ('Pct', 'NFLX')])

IndCols: [('Pct', 'SPY')], depCol ('Pct', 'AAPL'), cols [('Pct', 'SPY'), ('Pct', 'AAPL')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'AMZN'), cols [('Pct', 'SPY'), ('Pct', 'AMZN')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'FB'), cols [('Pct', 'SPY'), ('Pct', 'FB')]
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the follo

Beta 0                                            Beta 1  \
                AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                       
2017-09-08  0.000571  0.000526  0.001193  0.000334  0.001179  1.441811   
2017-10-06 -0.000303 -0.000153  0.000594  0.000447  0.001267  1.503402   
2017-11-03  0.000244  0.000158  0.000180 -0.000183  0.000702  1.598633   
2017-12-01 -0.000200 -0.000047  0.000014 -0.000689 -0.000275  1.626267   
2017-12-29  0.000088 -0.000012  0.000016 -0.000016  0.000480  1.507791   

                                                    
                AMZN        FB      GOOG      NFLX  
Dt                                                  
2017-09-08  1.295735  1.342542  1.297185  1.964616  
2017-10-06  1.401517  1.488949  1.331702  2.117310  
2017-11-03  1.768333  1.645183  1.517038  2.037191  
2017-12-01  1.875025  1.619385  1.481683  2.106296  
2017-12-29  1.768438  1.456644  1.248957  1.875548

## By concatenating the pct_df and beta_df
### we wind up with CONSISTENT indexing for returns and sensitivities
### BUT these are the UNION of the dates, not the intersecton

## Probably SHOULD do the concatention:
- the dates of the sensitivities don't HAVE to contain the dates for which we want to do attribution
 - e.g., we may want to project forward from last regression

In [38]:
beta_df.loc["2017-11-03":,:].to_csv("/tmp/beta_df.csv")

In [11]:
concatTrans = DataFrameConcat( [ pct_df, beta_df ])
ret_and_beta_df = concatTrans.fit_transform(pd.DataFrame())
ret_and_beta_df.loc[:"2017-12-29",:].tail()
ret_and_beta_df.shape

Beta 0                                           Beta 1  \
                AAPL      AMZN        FB      GOOG     NFLX      AAPL   
Dt                                                                      
2017-12-22       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-26       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-27       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-28       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-29  0.000088 -0.000012  0.000016 -0.000016  0.00048  1.507791   

                                                         Pct            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2017-12-22       NaN       NaN       NaN       NaN  0.000000 -0.005448   
2017-12-26       NaN       NaN       NaN       NaN -0.025370  0.007190   
2017-12-27       NaN       NaN       NaN       NaN  0.000176  0.004674   
2017-12-28       NaN       NaN       NaN       NaN  0.002814  0.003248   
2017-12-29  1.768438  1.456644  1.248957  1.875548 -0.010814 -0.014021   

                                                    
                  FB      GOOG      NFLX       SPY  
Dt                                                  
2017-12-22 -0.001409 -0.003300  0.006998 -0.000262  
2017-12-26 -0.006828 -0.003188 -0.011477 -0.001196  
2017-12-27  0.009262 -0.006974 -0.008095  0.000487  
2017-12-28  0.001689 -0.001172  0.034740  0.002057  
2017-12-29 -0.008206 -0.001660 -0.003892 -0.003771

(4581, 16)

## Append the rolling betas to the prepared data

In [40]:
ret_and_beta_df.loc["2017-11-03":,:].to_csv("/tmp/ret_and_beta_df.csv")

## Test the higher level RegPipe object

In [12]:
rp = RegPipe( pct_df )
rp.indCols( [ idx["Pct", "SPY"] ] )
rp.regress( regStart, regEnd, regWindow, regStep)

IndCols: [('Pct', 'SPY')], depCol ('Pct', 'AAPL'), cols [('Pct', 'SPY'), ('Pct', 'AAPL')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'AMZN'), cols [('Pct', 'SPY'), ('Pct', 'AMZN')]
IndCols: [('Pct', 'SPY')], depCol ('Pct', 'FB'), cols [('Pct', 'SPY'), ('Pct', 'FB')]
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the following columns have naN:  ['FB']
Fit: the follo

In [13]:
rp.beta_df.tail()

Beta 0                                            Beta 1  \
                AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                       
2017-09-08  0.000571  0.000526  0.001193  0.000334  0.001179  1.441811   
2017-10-06 -0.000303 -0.000153  0.000594  0.000447  0.001267  1.503402   
2017-11-03  0.000244  0.000158  0.000180 -0.000183  0.000702  1.598633   
2017-12-01 -0.000200 -0.000047  0.000014 -0.000689 -0.000275  1.626267   
2017-12-29  0.000088 -0.000012  0.000016 -0.000016  0.000480  1.507791   

                                                    
                AMZN        FB      GOOG      NFLX  
Dt                                                  
2017-09-08  1.295735  1.342542  1.297185  1.964616  
2017-10-06  1.401517  1.488949  1.331702  2.117310  
2017-11-03  1.768333  1.645183  1.517038  2.037191  
2017-12-01  1.875025  1.619385  1.481683  2.106296  
2017-12-29  1.768438  1.456644  1.248957  1.875548

In [14]:
rp.beta_df.shape

(229, 10)

## Set the internal data of RegPipe to sensitivities and returns

In [16]:
rollAmount = 1
fillMethod = "ffill"

rp.attrib_setup(pct_df, rp.beta_df, rollAmount, fillMethod)

addConst


/home/ubuntu/Notebooks/trans/reg.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[colName] = val


## Perform the attribution using the RegPipe object

In [17]:
rp.attrib()
rp.retAttr_df.loc[:"2018-02-07",:].tail()

Contrib from 1                                         \
                     AAPL      AMZN        FB      GOOG     NFLX   
Dt                                                                 
2018-02-01       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-02       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-05       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-06       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-07       0.000088 -0.000012  0.000016 -0.000016  0.00048   

           Contrib from SPY                                         Predicted  \
                       AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                              
2018-02-01        -0.001712 -0.002007 -0.001654 -0.001418 -0.002129 -0.001624   
2018-02-02        -0.032824 -0.038499 -0.031711 -0.027190 -0.040831 -0.032736   
2018-02-05        -0.063060 -0.073961 -0.060921 -0.052235 -0.078440 -0.062972   
2018-02-06         0.029707  0.034842  0.028699  0.024607  0.036952  0.029795   
2018-02-07        -0.008180 -0.009594 -0.007902 -0.006775 -0.010175 -0.008092   

                                                       Error            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2018-02-01 -0.002019 -0.001638 -0.001434 -0.001649  0.003714 -0.039948   
2018-02-02 -0.038510 -0.031695 -0.027206 -0.040351 -0.010654  0.067251   
2018-02-05 -0.073972 -0.060905 -0.052251 -0.077961  0.037987  0.046034   
2018-02-06  0.034831  0.028715  0.024591  0.037432  0.011997  0.003184   
2018-02-07 -0.009605 -0.007886 -0.006792 -0.009695 -0.013316 -0.008456   

                                          
                  FB      GOOG      NFLX  
Dt                                        
2018-02-01  0.034812 -0.000480 -0.017699  
2018-02-02  0.017143 -0.020580  0.049254  
2018-02-05  0.013501  0.001797  0.028714  
2018-02-06 -0.006371 -0.001102  0.007640  
2018-02-07 -0.019797 -0.022840  0.005329

## Perform the attribution using low-level RegAttr object

## Prepare for Return Attribution

### Find the attributes with the sensitivities

In [74]:
rab = RegAttr(pct_df)
rab.setSens(beta_df)
betaAttrs = rab.sensAttrs('^Beta \d+$')
betaAttrs

['Beta 0', 'Beta 1']

## Roll the betas forward

## Concatenate pct_df (high frequency) and beta_df (low frequency)
## so that they have same index before rolling the low frequency beta

In [75]:
concatTrans = DataFrameConcat( [ pct_df, beta_df ])
ret_and_beta_df = concatTrans.fit_transform(pd.DataFrame())

ret_and_beta_df.loc[:"2017-12-29",:].tail()
ret_and_beta_df.shape



Beta 0                                           Beta 1  \
                AAPL      AMZN        FB      GOOG     NFLX      AAPL   
Dt                                                                      
2017-12-22       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-26       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-27       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-28       NaN       NaN       NaN       NaN      NaN       NaN   
2017-12-29  0.000088 -0.000012  0.000016 -0.000016  0.00048  1.507791   

                                                         Pct            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2017-12-22       NaN       NaN       NaN       NaN  0.000000 -0.005448   
2017-12-26       NaN       NaN       NaN       NaN -0.025370  0.007190   
2017-12-27       NaN       NaN       NaN       NaN  0.000176  0.004674   
2017-12-28       NaN       NaN       NaN       NaN  0.002814  0.003248   
2017-12-29  1.768438  1.456644  1.248957  1.875548 -0.010814 -0.014021   

                                                    
                  FB      GOOG      NFLX       SPY  
Dt                                                  
2017-12-22 -0.001409 -0.003300  0.006998 -0.000262  
2017-12-26 -0.006828 -0.003188 -0.011477 -0.001196  
2017-12-27  0.009262 -0.006974 -0.008095  0.000487  
2017-12-28  0.001689 -0.001172  0.034740  0.002057  
2017-12-29 -0.008206 -0.001660 -0.003892 -0.003771

(4569, 16)

In [76]:
beta_r_pl = make_pipeline( GenSelectAttrsTransformer(betaAttrs),
                            ShiftTransformer(1),
                            FillNullTransformer(method="ffill"),
                            GenRenameAttrsTransformer(lambda col: col + ' rolled fwd', level=0)
                         )
beta_rolled_df = beta_r_pl.fit_transform(ret_and_beta_df)
beta_rolled_df.tail()

transform: pandas version <= 0.20.


Beta 0 rolled fwd                                         \
                        AAPL      AMZN        FB      GOOG     NFLX   
Dt                                                                    
2018-02-13          0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-14          0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-15          0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-16          0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-20          0.000088 -0.000012  0.000016 -0.000016  0.00048   

           Beta 1 rolled fwd                                          
                        AAPL      AMZN        FB      GOOG      NFLX  
Dt                                                                    
2018-02-13          1.507791  1.768438  1.456644  1.248957  1.875548  
2018-02-14          1.507791  1.768438  1.456644  1.248957  1.875548  
2018-02-15          1.507791  1.768438  1.456644  1.248957  1.875548  
2018-02-16          1.507791  1.768438  1.456644  1.248957  1.875548  
2018-02-20          1.507791  1.768438  1.456644  1.248957  1.875548

### Append the rolled betas to the regression results

In [77]:
rat = RegAttr(ret_and_beta_df)
rat.setSens(beta_rolled_df)

### Find the columns for: 
#### independent variables
#### dependent variables
#### sensitivities

In [78]:
indCols = [ ("Pct", "1"), ("Pct", "SPY")]
indCols

sensAttrs = rat.sensAttrs('^Beta \d+ rolled fwd$')
sensAttrs

depTickers = rat.depTickersFromSensAttrs(sensAttrs )
depTickers
depCols = [ ("Pct", t) for t in depTickers ]
depCols



[('Pct', '1'), ('Pct', 'SPY')]

['Beta 0 rolled fwd', 'Beta 1 rolled fwd']

['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX']

[('Pct', 'AAPL'),
 ('Pct', 'AMZN'),
 ('Pct', 'FB'),
 ('Pct', 'GOOG'),
 ('Pct', 'NFLX')]

## Add constant (for interecept return) column

In [79]:
rat.addConst(("Pct", "1"), 1)

addConst


### Perform the return attribution

In [80]:
retAttr_df =rat.retAttrib(
            indCols,
            depCols)


In [82]:
retAttr_df.loc[:"2018-02-07",:].tail()

Contrib from 1                                         \
                     AAPL      AMZN        FB      GOOG     NFLX   
Dt                                                                 
2018-02-01       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-02       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-05       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-06       0.000088 -0.000012  0.000016 -0.000016  0.00048   
2018-02-07       0.000088 -0.000012  0.000016 -0.000016  0.00048   

           Contrib from SPY                                         Predicted  \
                       AAPL      AMZN        FB      GOOG      NFLX      AAPL   
Dt                                                                              
2018-02-01        -0.001712 -0.002007 -0.001654 -0.001418 -0.002129 -0.001624   
2018-02-02        -0.032824 -0.038499 -0.031711 -0.027190 -0.040831 -0.032736   
2018-02-05        -0.063060 -0.073961 -0.060921 -0.052235 -0.078440 -0.062972   
2018-02-06         0.029707  0.034842  0.028699  0.024607  0.036952  0.029795   
2018-02-07        -0.008180 -0.009594 -0.007902 -0.006775 -0.010175 -0.008092   

                                                       Error            \
                AMZN        FB      GOOG      NFLX      AAPL      AMZN   
Dt                                                                       
2018-02-01 -0.002019 -0.001638 -0.001434 -0.001649  0.003714 -0.039948   
2018-02-02 -0.038510 -0.031695 -0.027206 -0.040351 -0.010654  0.067251   
2018-02-05 -0.073972 -0.060905 -0.052251 -0.077961  0.037987  0.046034   
2018-02-06  0.034831  0.028715  0.024591  0.037432  0.011997  0.003184   
2018-02-07 -0.009605 -0.007886 -0.006792 -0.009695 -0.013316 -0.008456   

                                          
                  FB      GOOG      NFLX  
Dt                                        
2018-02-01  0.034812 -0.000480 -0.017699  
2018-02-02  0.017143 -0.020580  0.049254  
2018-02-05  0.013501  0.001797  0.028714  
2018-02-06 -0.006371 -0.001102  0.007640  
2018-02-07 -0.019797 -0.022840  0.005329

In [24]:
retAttr_df.loc["2017-05-05":,:].to_csv("/tmp/retAttr.csv")

In [ ]:
gd.save_data(retAttr_df, "retattr_df.pkl")